In [4]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, Normalizer, StandardScaler, MinMaxScaler

from mlxtend.feature_selection import ColumnSelector

In [2]:
dataset_alpha = pd.read_csv('/Users/leewind/Projects/leewind/tianchi_O2O_predict/data_preprocessed_2/ProcessDataSet1.csv')

In [3]:
dataset_beta = pd.read_csv('/Users/leewind/Projects/leewind/tianchi_O2O_predict/data_preprocessed_2/ProcessDataSet2.csv')

In [6]:
continous = [
    'Coupon_id','Distance','discount_rate','weekday', 'day', 'u2', 'u3', 'u19',
    'u1', 'u4', 'u5', 'u25', 'u20', 'u6', 'u7', 'u8', 'u9', 'u10',
    'u11', 'u21', 'u22', 'u23', 'u24', 'u45', 'u27', 'u28', 'u32',
    'u47', 'u33', 'u34', 'u35', 'u36', 'u37', 'discount_type', 'u41',
    'u42', 'u43', 'u44', 'u48', 'u49', 'm0', 'm1', 'm2', 'm3', 'm4',
    'm7', 'm5', 'm6', 'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14',
    'm15', 'm18', 'm19', 'm20', 'm21', 'm22', 'm23', 'c1', 'c2', 'c3',
    'c4', 'c5', 'c6', 'c8', 'c9', 'c10', 'c11', 'c12', 'um1', 'um2',
    'um3', 'um4', 'um5', 'um6', 'um7', 'um8', 'um9', 'um10', 'um11',
    'um12', 'o1', 'o2', 'o17', 'o18', 'o3', 'o4', 'o5', 'o6', 'o7',
    'o8', 'o9', 'o10', 'o11', 'o12', 'o13', 'o14', 'o15', 'o16',
    'on_u1', 'on_u2', 'on_u3', 'on_u4', 'on_u5', 'on_u6', 'on_u7',
    'on_u8', 'on_u9', 'on_u10', 'on_u11', 'on_u12', 'on_u13'
]

In [5]:
dataset_alpha.columns.values


'User_id', 'Merchant_id', 'Discount_rate', 'Date_received', 'discount_rate_x', 'discount_rate_y','Date', 'Coupon_id', 'label'

array(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'discount_rate_x', 'discount_rate_y',
       'discount_rate', 'label', 'weekday', 'day', 'u2', 'u3', 'u19',
       'u1', 'u4', 'u5', 'u25', 'u20', 'u6', 'u7', 'u8', 'u9', 'u10',
       'u11', 'u21', 'u22', 'u23', 'u24', 'u45', 'u27', 'u28', 'u32',
       'u47', 'u33', 'u34', 'u35', 'u36', 'u37', 'discount_type', 'u41',
       'u42', 'u43', 'u44', 'u48', 'u49', 'm0', 'm1', 'm2', 'm3', 'm4',
       'm7', 'm5', 'm6', 'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14',
       'm15', 'm18', 'm19', 'm20', 'm21', 'm22', 'm23', 'c1', 'c2', 'c3',
       'c4', 'c5', 'c6', 'c8', 'c9', 'c10', 'c11', 'c12', 'um1', 'um2',
       'um3', 'um4', 'um5', 'um6', 'um7', 'um8', 'um9', 'um10', 'um11',
       'um12', 'o1', 'o2', 'o17', 'o18', 'o3', 'o4', 'o5', 'o6', 'o7',
       'o8', 'o9', 'o10', 'o11', 'o12', 'o13', 'o14', 'o15', 'o16',
       'on_u1', 'on_u2', 'on_u3', 'on_u4', 'on_u5', 'on_u6', 'on_u7',
 

In [8]:
features_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('continuous', Pipeline([
            ('extract', ColumnSelector(continous)),
            ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
#             ('scale', MinMaxScaler()),
            ('normalize', Normalizer())
        ])),
#         ('fields', Pipeline([
#             ('extract', ColumnSelector(fields)),
#             ('imputer', SimpleImputer(missing_values=np.nan,  strategy='most_frequent')),
#             ('one_hot', OneHotEncoder(categories='auto')),
#             ('to_dense', DenseTransformer())
#         ])),
    ])),
])

features_pipeline.fit(dataset_beta, dataset_beta.label.values.ravel())

train_dataset_x = features_pipeline.transform(dataset_beta)
train_dataset_y = dataset_beta.label.values.ravel()

valid_dataset_x = features_pipeline.transform(dataset_alpha)
valid_dataset_y = dataset_alpha.label.values.ravel()

In [9]:
selector_model = XGBClassifier(max_depth=3, n_estimators=20, random_state=0)
selector_model.fit(train_dataset_x, train_dataset_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=20,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [11]:
feature_selector = []
for index, value in enumerate(selector_model.feature_importances_):
    if value > 0:
        feature_selector.append((continous[index], value))

feature_selector

# selector_model.feature_importances_

[('Coupon_id', 0.021428572),
 ('Distance', 0.035714287),
 ('u3', 0.021428572),
 ('u5', 0.007142857),
 ('u20', 0.007142857),
 ('u28', 0.014285714),
 ('u35', 0.057142857),
 ('u43', 0.021428572),
 ('m2', 0.021428572),
 ('m10', 0.014285714),
 ('m13', 0.071428575),
 ('m23', 0.007142857),
 ('c1', 0.007142857),
 ('c2', 0.057142857),
 ('c5', 0.05),
 ('c10', 0.05),
 ('um7', 0.12857144),
 ('um10', 0.028571429),
 ('um11', 0.1),
 ('um12', 0.014285714),
 ('o1', 0.007142857),
 ('o4', 0.028571429),
 ('o5', 0.15714286),
 ('o6', 0.028571429),
 ('o7', 0.042857144)]